[Weekopdracht 6 2022](https://community-challenge.netlify.app/) - &copy; Paul Schouten 2022

# Is it in Stock?

Al vele jaren gaat Klaas naar de supermarkt bij hem om de hoek. Hij doet er niet alleen boodschappen voor zichzelf, maar ook voor zijn mindervalide buurvrouw. Elke dinsdag krijgt hij een boodschappenlijstje mee en brengt hij de boodschappen weer netjes bij haar thuis af.
Klaas heeft echter recent een extra baan gekregen en wil dit proces optimaliseren om zo meer tijd te besparen. Aan de hand van de volgende opdrachten ga jij hem helpen zo efficiënt mogelijk de boodschappen te doen.

<div style="font-size: 2em; text-align: center;">★</div>

Adrie heeft het boodschappenlijstje voor hem klaargelegd. Zijn alle producten op het lijstje ook te vinden in de supermarkt? Het assortiment is [hier](assortiment.json) te vinden.

Uitwerking: Eerst worden de JSON files ingelezen. Deze worden omgezet naar een list-of-named-tuples. Dit is (wellicht) handiger voor de vervolgstappen. Helaas blijkt dat de items op het boodschappenlijstje niet helemaal overeenkomen met de items in het assortiment. Om toch de producten te kunnen vinden in de winkel, wordt gebruikt gemaakt van een _fuzzy search_. Bij deze eerste opdracht wordt alleen gekeken of een product in het assortiment te vinden is.


In [1]:
import json
from thefuzz import process
import collections


# Inlezen json files
with open('lijstje.json') as f:
    boodschappenlijst = json.load(f)["lijstje"]
    
with open('assortiment.json') as f:
    assortiment = json.load(f)["producten"]

    
# boodschap is een _namedtuple_, voorbereiden op komende opdrachten    
boodschap = collections.namedtuple(
    'boodschap', 
    ['product', 'hoeveelheid', 'eenheid', 'aantal', 'subtotaal', 'schap'],
    defaults = (None, None, None, None, 999, None))
assorti = collections.namedtuple(
    'assorti', 
    ['product', 'schap', 'gewicht', 'eenheid', 'prijs'])

# Omzetten list-van-dicts naar list-van-namedtuple
lijstje = [ boodschap(**item) for item in boodschappenlijst ]
assortiment = [ assorti(**item) for item in assortiment ]

# Lijst met producten
producten = [ product.product for product in assortiment ]

# Loop over de boodschappenlijst
# Helaas geen exacte match tussen boodschappenlijst en assortiment,
#     --> thefuzz voor ongeveer matchen met strings
# Iedere item van het lijstje wordt opgezocht in het assortiment,
#     bij een match wordt het product toegevoegd aan gevonden_boodschappen
gevonden_boodschappen = []
for item in lijstje:
    print(f'Zoeken naar {item.product}: ')
    
    # Fuzzy string search, geeft hoogste score terug als 
    # tuple (string, score)
    gevonden = process.extract(item.product, producten)
    
    for vondst, score in gevonden:
        if score >= 90:
            # Goede match met gezocht item
            print(f'    Gevonden keuze: {vondst}')
            gevonden_boodschappen.append(item.product)

# Als alle boodschappen gevonden zijn, moet de set van gevonden_boodschappen
# even lang zijn als het (boodschappen)lijstje
print('')
if len(lijstje) == len(set(gevonden_boodschappen)):
    print('Alle boodschappen zijn gevonden!')
else:
    print('Niet alle boodschappen zijn gevonden...')


Zoeken naar melk: 
    Gevonden keuze: melk (literpak)
    Gevonden keuze: melk (groot)
    Gevonden keuze: karnemelk
Zoeken naar gildekorn: 
    Gevonden keuze: gildekorn
Zoeken naar cassis: 
    Gevonden keuze: cassis
Zoeken naar milde kwark: 
    Gevonden keuze: milde kwark
Zoeken naar mars: 
    Gevonden keuze: mars
Zoeken naar paprika chips: 
    Gevonden keuze: chips (paprika)
Zoeken naar eieren: 
    Gevonden keuze: eieren (6-pak)
    Gevonden keuze: eieren (dozijn)

Alle boodschappen zijn gevonden!


<div style="font-size: 2em; text-align: center;">★★</div>

Klaas krijgt altijd 15 euro mee om de boodschappen mee te doen. Heeft Klaas hier genoeg aan? De prijzen zijn te vinden in het [assortiment](assortiment.json).

Uitwerking: Het boodschappenlijstje wordt opnieuw bekeken, voor ieder product wat erop staat wordt opgezocht wat de prijs per stuk is. Het benodige aantal wordt ook uitgerekend, dit is de gewenste _hoeveelheid_ gedeeld door het _gewicht_. Het aantal wordt naar boven afgrond. Immers, als je 8 eieren wilt hebben, dan heb je er niet genoeg aan 6...


In [2]:
import math

# Eerst alle booschappen vinden in het assortiment,
# nu ook het benodigde aantal en het subtotaal uitrekenen
def boodschappen_zoeken(lijstje):
    gevonden_boodschappen = []
    for item in lijstje:
        print(f'Zoeken naar {item.product}, hoeveelheid: {item.hoeveelheid} {item.eenheid} ')

        # Fuzzy string search, geeft scores terug als 
        # tuple (string, score)
        gevonden = process.extract(item.product, producten)

        # Loop over ieder gevonden item in het assortiment
        beste_keuze = boodschap()

        for vondst, score in gevonden:
            # Bij een score van 90 worden goede producten gevonden. 
            # Gelukkig is de karnemelk duur...
            if score >= 90:
                # Goede match met gezocht product
                print(f'    Gevonden: {vondst} ', end='')
                assorti_keuze = [ product for product in assortiment if product.product == vondst ]

                # Uitzoeken hoeveel uit het assortiment te kopen,
                # afronden naar boven
                # subtotaal uitrekenen
                aantal_nodig = math.ceil(item.hoeveelheid / assorti_keuze[0].gewicht)
                subtotaal = aantal_nodig * assorti_keuze[0].prijs
                print(f'{aantal_nodig} x {assorti_keuze[0].prijs} = {subtotaal}', end=' ')

                if subtotaal < beste_keuze.subtotaal:
                    beste_keuze = boodschap(
                                        assorti_keuze[0].product, 
                                        assorti_keuze[0].gewicht,
                                        item.eenheid,
                                        aantal_nodig,
                                        subtotaal,
                                        assorti_keuze[0].schap)
                    # Er is een nieuwe beste keus
                    print('*')
                else:
                    # Naar de volgende regel
                    print('')

        # De beste keuze voor item uit lijstje toevoegen aan de gevonden boodschappen

        gevonden_boodschappen.append(beste_keuze)
        print()
    return gevonden_boodschappen
        
# Boodschappen opzoeken
gevonden_boodschappen = boodschappen_zoeken(lijstje)

# Nu het totaalbedrag uitrekenen
totaal_prijs = [ item.subtotaal for item in gevonden_boodschappen ]
totaal = sum(totaal_prijs)

print(f'Klaas moet €{totaal:.2f} betalen aan de kassa.', end=' ')
if totaal <= 15:
    print('Dit is minder dan €15, Klaas komt uit met het bedrag')
else:
    print(f'Klaas zal €{totaal-15:.2f} moeten bijbetalen...')

Zoeken naar melk, hoeveelheid: 2.4 liter 
    Gevonden: melk (literpak) 3 x 1.19 = 3.57 *
    Gevonden: melk (groot) 1 x 2.24 = 2.24 *
    Gevonden: karnemelk 3 x 1.32 = 3.96 

Zoeken naar gildekorn, hoeveelheid: 2 stuks 
    Gevonden: gildekorn 2 x 1.87 = 3.74 *

Zoeken naar cassis, hoeveelheid: 1 liter 
    Gevonden: cassis 1 x 1.37 = 1.37 *

Zoeken naar milde kwark, hoeveelheid: 500 gram 
    Gevonden: milde kwark 1 x 1.19 = 1.19 *

Zoeken naar mars, hoeveelheid: 1 multipack 
    Gevonden: mars 1 x 2.9 = 2.9 *

Zoeken naar paprika chips, hoeveelheid: 1 zak 
    Gevonden: chips (paprika) 1 x 1.14 = 1.14 *

Zoeken naar eieren, hoeveelheid: 8 stuks 
    Gevonden: eieren (6-pak) 2 x 2.14 = 4.28 *
    Gevonden: eieren (dozijn) 1 x 3.68 = 3.68 *

Klaas moet €16.26 betalen aan de kassa. Klaas zal €1.26 moeten bijbetalen...


<div style="font-size: 2em; text-align: center;">★★★</div>

De buurvrouw is jarig en wilt graag een taart bakken. Aangezien ze niet zo van het weggooien is wilt ze altijd goed kijken naar de hoeveelheid ingredienten en niet teveel kopen. Hoeveel blijft er over van de ingrediënten voor het [taartrecept](recept.json), als de boodschappen zo gedaan worden met de minste verspilling?

Uitwerking: Eerst wordt het recept ingelezen. De ingrediënten worden omgezet naar producten die in de winkel te verkrijgen zijn. Er zal bijvoorbeeld de hoeveelheid in _gram_ omgezet worden naar _kg_. Dan worden er boodschappen gedaan, op dezelfde manier als bij de vorige opgave. Tenslotte wordt uitgerekend hoeveel er van de ingrediënten overblijft na het bakken van de taart.


In [3]:
import re


# Recept inlezen
with open('recept.json') as f:
    taartrecept = json.load(f)

    
# Recept omzetten naar boodschappenlijst
def parse_recept(recept):
    # Haal het recept uit elkaar
    # Bepaal voor ieder item
    # - Product
    # - Hoeveelheid
    # - Eenheid
    print('Recept: ')
    boodschappen = []
    for item in recept:
        # Opsplitsen. Eerst door de spatie
        hoeveel, product = item.split(' ')
        
        # Getal en eenheid splitsen (yay, regex!)
        # re.split geeft als eerste '', deze wordt weggelaten
        _, hoeveelheid, eenheid = re.split('(\d{1,3})', hoeveel)
        
        # Hoeveelheid als getal
        hoeveelheid = int(hoeveelheid)
        
        # Producten omzetten naar geschikte grootheden en eenheden
        if product == 'boter':
            product = 'boter ongezouten' # want wie gebruikt gezouten boter?
            if eenheid == 'g':
                eenheid = 'gram'
        
        if eenheid == '':
            eenheid = 'stuks'
            
        if product == 'melk':
            if eenheid == 'ml':
                eenheid = 'liter'
                hoeveelheid /= 1000
                
        if product == 'bloem':
            if eenheid == 'g':
                eenheid = 'kg'
                hoeveelheid /= 1000
                
        if product == 'appels':
            product = 'appel'
            if eenheid == 'stuks':
                eenheid = 'kg'
                hoeveelheid *= 0.15 # 1 appel is ca. 150 gram, 1e hit Google
        
        print(f'    {hoeveelheid} {eenheid} {product}')
        boodschappen.append(boodschap(
            product,
            hoeveelheid,
            eenheid))
        
    return boodschappen


# Boodschappen opzoeken aan de hand van het recept
boodschappen_taart = parse_recept(taartrecept["recept"]["ingredienten"])

# Boodschappen opzoeken in het assortiment
print('\nBoodschappen doen:')
gevonden_boodschappen_taart = boodschappen_zoeken(boodschappen_taart)

# Hulpfunctie om onderstaand leesbaar te houden,
# voegt product en eenheid samen
def samenvoegen_product(product):
    return f'{product.product.split()[0]}-{product.eenheid}'

# Omzetten voorraad en taartbenodigdheden naar een Counter 
voorraad = collections.Counter(
    { samenvoegen_product(item):item.hoeveelheid for item in gevonden_boodschappen_taart })
taart = collections.Counter(
    { samenvoegen_product(item):item.hoeveelheid for item in boodschappen_taart })

# Nu kan zonder itereren het verschil uitgerekend worden
over = voorraad - taart

print('Dit is er nog over na het bakken van de taart:')
for item in over:
    product, eenheid = item.split('-')
    hoeveelheid = over[item]
    print(f'    {hoeveelheid} {eenheid} {product}')

Recept: 
    80 gram boter ongezouten
    4 stuks eieren
    0.25 liter melk
    0.1 kg bloem
    0.6 kg appel

Boodschappen doen:
Zoeken naar boter ongezouten, hoeveelheid: 80 gram 
    Gevonden: boter ongezouten 1 x 1.46 = 1.46 *
    Gevonden: boter gezouten 1 x 1.49 = 1.49 

Zoeken naar eieren, hoeveelheid: 4 stuks 
    Gevonden: eieren (6-pak) 1 x 2.14 = 2.14 *
    Gevonden: eieren (dozijn) 1 x 3.68 = 3.68 

Zoeken naar melk, hoeveelheid: 0.25 liter 
    Gevonden: melk (literpak) 1 x 1.19 = 1.19 *
    Gevonden: melk (groot) 1 x 2.24 = 2.24 
    Gevonden: karnemelk 1 x 1.32 = 1.32 

Zoeken naar bloem, hoeveelheid: 0.1 kg 
    Gevonden: bloem 1 x 0.67 = 0.67 *

Zoeken naar appel, hoeveelheid: 0.6 kg 
    Gevonden: appel 1 x 2.19 = 2.19 *

Dit is er nog over na het bakken van de taart:
    120 gram boter
    2 stuks eieren
    0.75 liter melk
    0.9 kg bloem
    0.4 kg appel


## Bonus

Binnenin de supermarkt is de plek van de producten bekend. Met behulp van [Dijkstra's algoritme](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) kan je kijken wat de kortste route in de supermarkt is. Zo kan je de looproute optimaliseren en dus weer sneller thuis zijn.

Hieronder is een schematische weergave van de supermarkt toegevoegd. Bereken de korste route aan de hand van het boodschappenlijstje van de eerste opdracht. Uiteraard moet je langs elk schap van het product op je lijstje.

<img src="supermarkt.png" style="width: 75%; display: block; margin-left: auto; margin-right: auto;" />

Uitwerking: 
- Dijkstra's algoritme programmeren, wat is de kortste route vanaf de ingang naar de uitgang?
- Welke schappen moet er langs gegaan worden?
- In welke **permutatie** is de afstand in de winkel het kortst?

Als eerste wordt het algoritme van Dijkstra uitgewerkt. In plaats van een van de vele voorbeelden op internet te kopiëren, heb ik deze op basis van de beschijving van het algoritme zelf opgesteld. Op deze manier, inclusief de lay-out van de supermarkt op een stuk papier, hoofdrekenen en hoofdpijn, is er voor mij het meeste te leren...

In [4]:
# Grafentheorie: Edges verbinden Nodes
import collections


# Beschrijven lay-out supermarkt:
# list-of-list: van-naar-afstand
supermarkt = [
    ["ingang", "groente", 4],
    ["ingang", "brood", 6],
    ["groente", "brood", 8],
    ["groente", "snacks", 6],
    ["groente", "non-food", 12],
    ["groente", "koelschap", 5],
    ["koelschap", "snacks", 11],
    ["koelschap", "non-food", 12],
    ["koelschap", "diepvries", 8],
    ["koelschap", "bakproducten", 5],
    ["bakproducten", "non-food", 7],
    ["bakproducten", "diepvries", 8],
    ["bakproducten", "kassa", 5],
    ["bakproducten", "dranken", 6],
    ["dranken", "kassa", 6],
    ["brood", "snacks", 9],
    ["snacks", "non-food", 5],
    ["non-food", "diepvries", 3],
    ["diepvries", "kassa", 3],
    ["kassa", "uitgang", 5],
]


# Onderlinge samenhang bepalen:
# dicts met inhoud list, zie https://docs.python.org/3/library/collections.html#collections.defaultdict
# Hiermee kan via edges["node"] een lijst verkregen worden van de naastgelegen nodes

# Met een dict kunnen de namen gebruikt worden als index, dat leest gemakkelijker dan een index
# Bij de eerste keer refereren wordt een lege list aangemaakt
edges = collections.defaultdict(list)
afstanden = {}

# Ieder item uit de supermarkt in _edges_ en _afstanden_ zetten
for item in supermarkt:
    a, b, afstand = item
    # Edges verbinden twee kanten op: A->B en B->A
    edges[a].append(b)
    edges[b].append(a)
    afstanden[(a, b)] = afstand
    afstanden[(b, a)] = afstand


# Dijkstra algoritme 
def dijkstra(start_node, stop_node):
    # Er zijn geen nodes bezocht
    bezocht = []

    # Bijhouden via welke route de kortste totale afstand bereikt wordt
    # Totale afstand om tot een node te komen 
    # Als defaultdict, zodat bij de eerste keer opzoeken de default waarde inf is
    totale_afstand = collections.defaultdict(lambda: float('inf'))

    # De node met de kortste afstand VOOR deze node opslaan
    beste_keuze = {}

    totale_afstand[start_node] = 0

    # while loop opstarten
    huidige_node = start_node

    # Blijf herhalen totdat de stop_node bereikt is
    while huidige_node != stop_node:
        # Node toevoegen aan bezochte nodes
        bezocht.append(huidige_node)

        # Welke nodes zijn nog niet bezocht?
        nodes_bezoeken = [ n for n in edges[huidige_node] if n not in bezocht ]

        # Loop over iedere node in nodes
        for node in nodes_bezoeken:
            afstand = totale_afstand[huidige_node] + afstanden[(huidige_node, node)]

            if afstand < totale_afstand[node]:
                totale_afstand[node] = afstand
                beste_keuze[node] = huidige_node

        # Nodes zoeken (die nog niet bezocht zijn)
        # deze in de volgende loop beoordelen
        mogelijkheden = { node:values for node,values in totale_afstand.items() if node not in bezocht }

        # Volgende loop: de node met de kleinste afstand beoordelen
        huidige_node = min(mogelijkheden, key=mogelijkheden.get, default=None)

    # Afgelegde route: achteruit zoeken
    route = []
    vorige_node = stop_node
    while vorige_node != start_node:
        route.insert(0, vorige_node)
        vorige_node = beste_keuze[vorige_node]
    route.insert(0, start_node)

    return route, totale_afstand[stop_node], totale_afstand

# Bepaal de snelste route van de ingang naar de uitgang
route, afstand, totale_afstand = dijkstra("ingang", "uitgang")
print(f'Snelste route: {route}')
print(f'Afgelegde afstand: {afstand}')

Snelste route: ['ingang', 'groente', 'koelschap', 'bakproducten', 'kassa', 'uitgang']
Afgelegde afstand: 24


Om te bepalen welke schappen langs gegaan moeten worden, wordt de `set` van de boodschappen bepaald. 

In [5]:
schappen = set([ item.schap for item in gevonden_boodschappen ])

print(f'De volgende schappen moeten bezocht worden: {schappen}')

De volgende schappen moeten bezocht worden: {'snacks', 'bakproducten', 'koelschap', 'dranken', 'brood'}


Vanwege het _greedy_ karakter (iedere keer wordt de dichtbijzijnste node het eerst onderzocht) van Dijkstra's algoritme zal de beste volgorde in de volgorde van de gevonden nodes zijn:

In [6]:
volgorde = { node:values for node,values in totale_afstand.items() if node in schappen }

print(f'Ideale route door de supermarkt: {"->".join(volgorde.keys())}')

Ideale route door de supermarkt: brood->snacks->koelschap->bakproducten->dranken


Om dit te bewijzen worden alle mogelijke routes uitgeprobeerd. Met 5 verschillende schappen zijn er 5! (als in faculteit) = 120 mogelijkheden om langs de rekken te lopen. Gelukkig heeft Python een module om dit soort iteraties uit te voeren: `itertools`.

In [7]:
import itertools
import pprint


def bepaal_route_en_afstand(it):
    # Eerst uitpakken van iterable
    a, b, c, d, e = it
    route_testen = ["ingang", a, b, c, d, e, "uitgang"]
    route_afstand = 0
    
    for i in range(len(route_testen)-1):
        _, afstand, _ = dijkstra(route_testen[i], route_testen[i+1])
        route_afstand += afstand
        
    return route_afstand, route_testen

routes = list(map(bepaal_route_en_afstand, itertools.permutations(schappen)))

routes.sort()  

pprint.pprint(routes, width=100)

[(48, ['ingang', 'brood', 'snacks', 'koelschap', 'bakproducten', 'dranken', 'uitgang']),
 (53, ['ingang', 'brood', 'snacks', 'koelschap', 'dranken', 'bakproducten', 'uitgang']),
 (54, ['ingang', 'brood', 'snacks', 'bakproducten', 'koelschap', 'dranken', 'uitgang']),
 (54, ['ingang', 'snacks', 'brood', 'koelschap', 'bakproducten', 'dranken', 'uitgang']),
 (58, ['ingang', 'brood', 'snacks', 'dranken', 'bakproducten', 'koelschap', 'uitgang']),
 (58, ['ingang', 'brood', 'snacks', 'dranken', 'koelschap', 'bakproducten', 'uitgang']),
 (59, ['ingang', 'brood', 'koelschap', 'snacks', 'bakproducten', 'dranken', 'uitgang']),
 (59, ['ingang', 'brood', 'snacks', 'bakproducten', 'dranken', 'koelschap', 'uitgang']),
 (59, ['ingang', 'snacks', 'brood', 'koelschap', 'dranken', 'bakproducten', 'uitgang']),
 (60, ['ingang', 'koelschap', 'brood', 'snacks', 'bakproducten', 'dranken', 'uitgang']),
 (63, ['ingang', 'brood', 'koelschap', 'bakproducten', 'dranken', 'snacks', 'uitgang']),
 (63, ['ingang', 'bro

Er zit veel verschil in de afstand van alle mogelijke routes. De maximale afstand is 100, terwijl de minimale afstand 48 is. Dit is bij deze volgorde:

In [8]:
print(f'De optimale route is: {"->".join(routes[0][1])}')

De optimale route is: ingang->brood->snacks->koelschap->bakproducten->dranken->uitgang


En deze is inderdaad hetzelfde als bovenstaand 😀